In [1]:
from selenium import webdriver
import time, re

In [16]:
def lat_lon_head_to_url(lat, lon, heading):
    return "https://www.google.com/maps/?q={lat},{lon}&h={head}".format(lat=lat, lon=lon, head=heading)


pattern = r'\d*\.\d*h'
reg = re.compile(pattern)
# find between @ and first /
pattern2 = r'(@.*?/)'
reg2 = re.compile(pattern2)
def change_url_heading(heading):
    time.sleep(1)
    url = driver.current_url
    splitted = re.split(pattern, url, maxsplit=1)
    try:
        new_url = splitted[0] + str(heading) + 'h' + splitted[1]
    except IndexError:
        splitted = re.split(reg2, url, maxsplit=1)
        try:
            new_url = splitted[0] + splitted[1][:-1] + ',' + str(heading) + 'h/' + splitted[2]
        except IndexError:
            print(splitted)
            
    time.sleep(4)
    driver.get(new_url)

def selenium_to_url_sv(url, driver):
    driver.get(url)
    try:
        driver.find_element_by_css_selector(
            "button.section-hero-header-image-hero.widget-pane-fading.widget-pane-fade-in.section-hero-header-image-hero-clickable"
        ).click()
    except NoSuchElement:
        raise
    
def selenium_look_right_snap(heading, driver, path):
    right_heading = (heading + 90) % 360
    change_url_heading(right_heading)
    time.sleep(5)
    driver.save_screenshot(path)
    

def seleneium_look_left_snap(heading, driver, path):
    left_heading = (heading - 90) % 360
    change_url_heading(left_heading)
    time.sleep(5)
    driver.save_screenshot(path)
    time.sleep(1)
    
def snap_lat_long(driver, lat, lon, heading, folder):
    url = lat_lon_head_to_url(lat, lon, heading)
    try: 
        selenium_to_url_sv(url,  driver)
    except:
        with open(folder + 'error', 'w') as error_file:
            print('error at ', lat, lon)
            error_file.write('could not get street view')
    selenium_look_right_snap(heading, driver, folder + str(lat) + ',' + str(lon) + '_' + 'right.png')
 
    seleneium_look_left_snap(heading, driver, folder + str(lat) + ',' + str(lon) + '_' + 'left.png')
 

In [3]:
import pickle as pkl
with open("../brighton_graph.pkl", 'rb') as graphpkl:
    brighton_G = pkl.load(graphpkl)


In [4]:
import geopandas as gpd
streets = gpd.read_file("../data/brighton/brighton_streets.shp")

In [5]:
from shapely.geometry import GeometryCollection
from copy import deepcopy
def list_explode(gdf):
    rows = []
    old_rows = []
    for i, row in gdf.iterrows():
        mline = row['cut_geometry']
        if type(mline) == GeometryCollection:
            mline = mline.geoms
        new_rows = [deepcopy(row) for i in range(len(row['cut_geometry']))]
        for n, inline in enumerate(mline):
            new_rows[n]['geometry'] = inline
            old_rows.append(i)
        rows.extend(new_rows)
    new_gdf = gpd.GeoDataFrame(rows).reset_index(drop=True)
    new_gdf.crs = "EPSG:26919"
    return new_gdf


In [6]:
# Thank you Sean Gilles https://gist.github.com/sgillies/465156#file_cut.py
from shapely.geometry import LineString, Point
from copy import deepcopy
def cut(line, distance):
    # Cuts a line in two at a distance from its starting point
    if distance <= 0.0 or distance >= line.length:
        return [LineString(line)]
    coords = list(line.coords)
    for i, p in enumerate(coords):
        pd = line.project(Point(p))
        if pd == distance:
            return [
                LineString(coords[:i+1]),
                LineString(coords[i:])]
        if pd > distance:
            cp = line.interpolate(distance)
            return [
                LineString(coords[:i] + [(cp.x, cp.y)]),
                LineString([(cp.x, cp.y)] + coords[i:])]
        

def recursive_cut(line, distance):
    if [line] == cut(line, distance):
        return [line]
    else:
        segment, rest = cut(line, distance)
        return [segment] + recursive_cut(rest, distance)


In [7]:
streets = streets.to_crs("EPSG:26919")

In [8]:
def mls_to_list_safe(geom):
    if type(geom) == LineString:
        return [geom]
    elif (type(geom) == list and len(geom) == 1):
        return geom
    else:
        try:
            return geom.geoms
        except AttributeError:
            #list_of_geoms = list(geom)
            #[g for geom in list_of_geoms for g in geom]
            return list(geom)

streets['cut_geometry'] = streets.geometry.map(mls_to_list_safe)
streets = list_explode(streets)
streets['cut_geometry'] = streets.geometry.map(lambda x : recursive_cut(x,12))
streets_split = list_explode(streets)

In [9]:
streets_split['length_m'] = streets_split.length
streets_split.drop(inplace=True, columns=['length'])

streets_split = streets_split.to_crs("EPSG:4326")
streets_split.drop(inplace=True, columns=['cut_geometry'])

In [10]:
def midpoint(line):
    p0 = line.coords[0]
    p1 = line.coords[-1]
    x0, y0 = p0
    x1, y1 = p1
    return ((x0 + x1)/2, (y0 + y1)/2)

import math
def heading(line):
    # true north 0, east is 90
    # so 90 - arctan
    p0 = line.coords[0]
    p1 = line.coords[-1]
    x0, y0 = p0
    x1, y1 = p1
    
    seg = (x1 - x0, y1 - y0)
    angle = math.atan(seg[1]/seg[0]) * 180 / math.pi
    return ((90 - angle) % 360)
    

In [11]:
# beta reduce or whatever
streets_split['midpoint'] = streets_split.geometry.map(lambda x : midpoint(x))
streets_split['heading'] = streets_split.geometry.map(lambda x : int(heading(x)))

In [12]:
# used namedtuple convention
def process_row(driver, row):
    if row.length_m < 11.9:
        pass
    lon = row.midpoint[0]
    lat = row.midpoint[1]
    heading = row.heading
    snap_lat_long(driver, lat, lon, heading, "../data/streetphotos/")

    
    

In [18]:
len(streets_split[streets_split['length_m'] > 11.9])

16042

In [17]:
import os.path

driver = webdriver.Firefox()
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver.implicitly_wait(10)

from tqdm import tqdm
for row in tqdm(streets_split.itertuples()):
    lon = row.midpoint[0]
    lat = row.midpoint[1]
    try:
        if (os.path.isfile("../data/streetphotos/" + str(lat) + ',' + str(lon) + '_' + 'right.png' ) 
            and
            os.path.isfile("../data/streetphotos/" + str(lat) + ',' + str(lon) + '_' + 'left.png' )):
            print('skipped')
            continue
        else: 
            process_row(driver, row)
            time.sleep(15)
    except:
        raise
        with open("../data/streetphotos/" + 'error' + str(lat) + ',' + str(lon), 'w') as error_file:
            error_file.write('could not get street view')
        



0it [00:00, ?it/s]

1it [00:51, 51.95s/it]

2it [01:36, 49.69s/it]

KeyboardInterrupt: 